Before proceeding, please ensure that your notebook is connected to a GPU runtime.

In [1]:
%cd /content
!git clone https://github.com/atakan-topaloglu/2d-gaussian-splatting --recursive
!pip install -q plyfile

/content
Cloning into '2d-gaussian-splatting'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 238 (delta 110), reused 77 (delta 77), pack-reused 66 (from 1)
Receiving objects: 100% (238/238), 9.39 MiB | 6.79 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Submodule 'submodules/diff-surfel-rasterization' (https://github.com/hbb1/diff-surfel-rasterization.git) registered for path 'submodules/diff-surfel-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/2d-gaussian-splatting/submodules/diff-surfel-rasterization'...
remote: Enumerating objects: 697, done.        
remote: Counting objects: 100% (287/287), done.        
remote: Compressing objects: 100% (33/33), done.        
remote: Total 697 (delta 272), reused 254 (delta 254), pack-reused 410 (from 1)        
Receivin

In [2]:
%cd /content/2d-gaussian-splatting/

/content/2d-gaussian-splatting


In [3]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda' #ensure that the CUDA path is correct
!pip install ninja
#!pip install submodules/simple-knn
!pip install submodules/diff-surfel-rasterization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 11.3 MB/s eta 0:00:00
Processing ./submodules/diff-surfel-rasterization
  Preparing metadata (setup.py) ... done
  Created wheel for diff_surfel_rasterization: filename=diff_surfel_rasterization-0.0.1-cp312-cp312-linux_x86_64.whl size=3451452 sha256=afa192973a5d23d199b62c2bd3dd2d9341cc12b0d13dd2b94e63beecf285899f
  Stored in directory: /root/.cache/pip/wheels/b4/b3/55/b917b45654830b0f9349c0cf1ed78523fa7964ed3a994db822
Successfully built diff_surfel_rasterization


In [4]:
%cd /content/2d-gaussian-splatting/submodules/simple-knn

# Add #include <float.h> to the top of the CUDA source file
!sed -i '1i #include <float.h>' simple_knn.cu  || sed -i '1i #include <float.h>' simple-knn.cu

# Install in editable mode
!pip install -q -e .


/content/2d-gaussian-splatting/submodules/simple-knn
  Preparing metadata (setup.py) ... done


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/2d-gaussian-splatting/

/content/2d-gaussian-splatting


For Colab to access your COLMAP processed data, please copy the COLMAP output to your Google Drive and enter the full COLMAP source path (e.g. /content/drive/MyDrive/COLMAP_out)

In [7]:
COLMAP_PATH = str(input('Please enter COLMAP output path:'))

Please enter COLMAP output path:/content/drive/MyDrive/ADL4CV/Datasets/tandt_db/tandt/train


### Bug fix for training to work


*   replace colormap function in utils/general_utils.py with:

```
def colormap(data, cmap='turbo'):
    import matplotlib.pyplot as plt
    import numpy as np

    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    im = ax.imshow(data, cmap=cmap)
    fig.canvas.draw()

    buf = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf = buf.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    data = buf[..., 1:]  # drop alpha, keep RGB

    plt.close(fig)
    return data
```

* changed the whole training_report function in train.py



```
def training_report(tb_writer, iteration, Ll1, loss, l1_loss, elapsed, testing_iterations, scene: Scene, renderFunc, renderArgs):
    import numpy as np
    import torch
    from utils.general_utils import colormap

    # Scalars
    if tb_writer:
        tb_writer.add_scalar('train_loss_patches/reg_loss', Ll1.item(), iteration)
        tb_writer.add_scalar('train_loss_patches/total_loss', loss.item(), iteration)
        tb_writer.add_scalar('iter_time', elapsed, iteration)
        tb_writer.add_scalar('total_points', scene.gaussians.get_xyz.shape[0], iteration)

    # Report test and samples of training set
    if iteration in testing_iterations:
        torch.cuda.empty_cache()
        validation_configs = (
            {'name': 'test', 'cameras': scene.getTestCameras()},
            {
                'name': 'train',
                'cameras': [
                    scene.getTrainCameras()[idx % len(scene.getTrainCameras())]
                    for idx in range(5, 30, 5)
                ],
            },
        )

        for config in validation_configs:
            if config['cameras'] and len(config['cameras']) > 0:
                l1_test = 0.0
                psnr_test = 0.0

                for idx, viewpoint in enumerate(config['cameras']):
                    render_pkg = renderFunc(viewpoint, scene.gaussians, *renderArgs)

                    # Rendered RGB in [0,1], ground truth
                    image = torch.clamp(render_pkg["render"], 0.0, 1.0)
                    gt_image = torch.clamp(viewpoint.original_image.to("cuda"), 0.0, 1.0)

                    if tb_writer and (idx < 5):
                        # ---------- DEPTH IMAGE ----------
                        depth = render_pkg["surf_depth"]  # torch tensor
                        norm = depth.max()
                        depth = depth / (norm + 1e-8)

                        # depth: (1, H, W) -> numpy HWC via colormap -> torch NCHW
                        depth_np = depth.detach().cpu().numpy()[0]   # (H, W)
                        depth_rgb = colormap(depth_np, cmap='turbo') # (H, W, 3), uint8
                        depth_rgb = np.array(depth_rgb, copy=True)
                        depth_t = torch.from_numpy(depth_rgb).permute(2, 0, 1).unsqueeze(0)  # (1, 3, H, W)
                        depth_t = depth_t.float() / 255.0

                        # ---------- RENDERED IMAGE ----------
                        img_np = image.detach().cpu().numpy()[0]     # (3, H, W) or (H, W, 3) depending on code
                        if img_np.ndim == 3 and img_np.shape[0] == 3:
                            # (C, H, W) -> (H, W, C)
                            img_np = np.transpose(img_np, (1, 2, 0))
                        img_rgb = colormap(img_np)                   # (H, W, 3)
                        img_rgb = np.array(img_rgb, copy=True)
                        image_t = torch.from_numpy(img_rgb).permute(2, 0, 1).unsqueeze(0)  # (1, 3, H, W)
                        image_t = image_t.float() / 255.0

                        tb_writer.add_images(
                            config['name'] + "_view_{}/depth".format(viewpoint.image_name),
                            depth_t,
                            global_step=iteration,
                        )
                        tb_writer.add_images(
                            config['name'] + "_view_{}/render".format(viewpoint.image_name),
                            image_t,
                            global_step=iteration,
                        )

                        # ---------- NORMALS / ALPHA / DIST ----------
                        try:
                            rend_alpha = render_pkg['rend_alpha']          # expect (1,1,H,W) or similar
                            rend_normal = render_pkg["rend_normal"] * 0.5 + 0.5
                            surf_normal = render_pkg["surf_normal"] * 0.5 + 0.5

                            tb_writer.add_images(
                                config['name'] + "_view_{}/rend_normal".format(viewpoint.image_name),
                                rend_normal[None],
                                global_step=iteration,
                            )
                            tb_writer.add_images(
                                config['name'] + "_view_{}/surf_normal".format(viewpoint.image_name),
                                surf_normal[None],
                                global_step=iteration,
                            )
                            tb_writer.add_images(
                                config['name'] + "_view_{}/rend_alpha".format(viewpoint.image_name),
                                rend_alpha[None],
                                global_step=iteration,
                            )

                            # rend_dist: tensor -> numpy -> colormap -> torch NCHW
                            rend_dist = render_pkg["rend_dist"]            # torch tensor
                            dist_np = rend_dist.detach().cpu().numpy()[0]  # (H, W)
                            dist_rgb = colormap(dist_np)                   # (H, W, 3)
                            dist_rgb = np.array(dist_rgb, copy=True)
                            dist_t = torch.from_numpy(dist_rgb).permute(2, 0, 1).unsqueeze(0)
                            dist_t = dist_t.float() / 255.0

                            tb_writer.add_images(
                                config['name'] + "_view_{}/rend_dist".format(viewpoint.image_name),
                                dist_t,
                                global_step=iteration,
                            )
                        except Exception:
                            pass

                        # ---------- GROUND TRUTH ----------
                        if iteration == testing_iterations[0]:
                            tb_writer.add_images(
                                config['name'] + "_view_{}/ground_truth".format(viewpoint.image_name),
                                gt_image[None],
                                global_step=iteration,
                            )

                    # accumulate metrics
                    l1_test += l1_loss(image, gt_image).mean().double()
                    psnr_test += psnr(image, gt_image).mean().double()

                psnr_test /= len(config['cameras'])
                l1_test /= len(config['cameras'])
                print("\n[ITER {}] Evaluating {}: L1 {} PSNR {}".format(iteration, config['name'], l1_test, psnr_test))
                if tb_writer:
                    tb_writer.add_scalar(config['name'] + '/loss_viewpoint - l1_loss', l1_test, iteration)
                    tb_writer.add_scalar(config['name'] + '/loss_viewpoint - psnr', psnr_test, iteration)

        torch.cuda.empty_cache()

```







In [9]:
!python train.py -s $COLMAP_PATH

2025-11-11 09:46:33.193828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762854393.215182    9974 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762854393.221671    9974 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762854393.238385    9974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762854393.238411    9974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762854393.238415    9974 computation_placer.cc:177] computation placer alr

Training is complete, to use render.py, please enter the desired model_path located in your Drive: /content/drive/MyDrive/2d_gaussian_splatting/output/{10 digit UUID}: (e.g: /content/drive/MyDrive/2d_gaussian_splatting/output/8d812792-1)

In [14]:
MODEL_PATH = str(input('Please enter the model path:'))

Please enter the model path:/content/drive/MyDrive/2d_gaussian_splatting/output/158b3583-2


In [12]:
#!pip3 install open3d==0.18.0 mediapy==1.1.2 lpips==0.1.4 scikit-image==0.21.0 tqdm==4.66.2 trimesh==4.3.2 opencv-python
!pip3 install open3d mediapy lpips scikit-image tqdm trimesh opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [15]:
!python render.py -m $MODEL_PATH -s $COLMAP_PATH

Looking for config file in /content/drive/MyDrive/2d_gaussian_splatting/output/158b3583-2/cfg_args
Config file found: /content/drive/MyDrive/2d_gaussian_splatting/output/158b3583-2/cfg_args
Rendering /content/drive/MyDrive/2d_gaussian_splatting/output/158b3583-2
Loading trained model at iteration 30000
Reading camera 301/301
Loading Training Cameras
Loading Test Cameras
export training images ...
reconstruct radiance fields: 301it [00:08, 33.92it/s]
The estimated bounding radius is 1.63
Use at least 3.26 for depth_trunc
export images: 301it [02:41,  1.87it/s]
export mesh ...
reconstruct radiance fields: 301it [00:07, 37.80it/s]
The estimated bounding radius is 1.63
Use at least 3.26 for depth_trunc
Running tsdf volume integration ...
voxel_size: 0.003184356764539623
sdf_trunc: 0.015921783822698116
depth_truc: 3.260781326888574
TSDF integration progress: 22it [00:27,  1.23s/it]^C


Training and rendering is complete, please navigate to your drive and see the model outputs under the directory "2d_gaussian_splatting/".